In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.gray()

<Figure size 432x288 with 0 Axes>

In [2]:
import sys
sys.path.append('..')

In [3]:
full_archive = np.load('../data_folder/ct/full_archive.npz', allow_pickle = True)

In [4]:
input_images = full_archive["image"]

In [10]:
import prox_tv
from skimage.transform import radon, iradon, iradon_sart
from PIL import Image, ImageDraw

def get_mask(size):
    mask = Image.new('L', size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((-1, -1) + size, fill=1)
    mask = np.array(mask)
    return mask

def make_window(img, a, b):
    assert(a < b)
    img = np.minimum(255, np.maximum(0, (img - a) / (b - a) * 255))
    return img

def make_fbp(img, size):
    img = np.array(img)
    theta = np.linspace(0., 360., size, endpoint=False)
    sino = radon(img, theta=theta, circle=True)
    return iradon(sino, theta=theta, circle=True)

img = make_window(input_images[0], -1024, 1024)
fbp = make_fbp(img, 64)
tv1 = prox_tv.tv1_2d(fbp, 0.1)

In [16]:
import skimage.measure

def normalize(src):
    mi = np.min(src)
    ma = np.max(src)
    return (src - mi) / (ma - mi)

def compare(src, result, sum_):
    # src = normalize(src)
    # result = normalize(result)
    psnr = skimage.measure.compare_psnr(src, result)
    ssim = skimage.measure.compare_ssim(src, result)
    sum_[0] += psnr
    sum_[1] += ssim

sum_fbp = {64:[0., 0.], 128:[0., 0.], 256:[0., 0.], 512:[0., 0.]}
sum_sart = {64:[0, 0], 128:[0, 0], 256:[0, 0], 512:[0, 0]}
sum_tv1 = {64:[0, 0], 128:[0, 0], 256:[0, 0], 512:[0, 0]}

for i, img in enumerate(input_images):
    print("step {}".format(i))
    if img.shape != (512, 512):
        continue
    img = make_window(img, -1024, 1024)
    img = normalize(img * get_mask(img.shape))
    #plt.imshow(img)
    #plt.show()
    for size in [64, 128, 256, 512]:
        print("size {}".format(size), end="\r")
        theta = np.linspace(0., 360., size, endpoint=False)
        sino = radon(img, theta=theta, circle=True)
        fbp = iradon(sino, theta=theta, circle=True)
        #plt.imshow(fbp)
        #plt.show()
        sart = iradon_sart(sino, theta=theta)
        tv1 = prox_tv.tv1_2d(fbp, 0.1)
        #plt.imshow(tv1)
        #plt.show()
        compare(img, fbp, sum_fbp[size])
        compare(img, sart, sum_sart[size])
        compare(img, tv1, sum_tv1[size])
    for target in [[sum_fbp, "FBP"], [sum_sart, "SART"], [sum_tv1, "TV1"]]:
        for j in sorted(target[0].keys()):
            print("{}-{} PSNR:{} SSIM:{}".format(target[1], j, target[0][j][0] / (i+1), target[0][j][1] / (i+1)))

step 0
FBP-64 PSNR:20.128395869 SSIM:0.476827566781
FBP-128 PSNR:24.762792783 SSIM:0.661051917788
FBP-256 PSNR:29.2072904364 SSIM:0.850846806372
FBP-512 PSNR:31.9316705508 SSIM:0.935258979129
SART-64 PSNR:24.3322021121 SSIM:0.788097513325
SART-128 PSNR:26.1244653158 SSIM:0.848550408042
SART-256 PSNR:27.5413939643 SSIM:0.89509196414
SART-512 PSNR:28.4859610775 SSIM:0.921782636768
TV1-64 PSNR:24.954450579 SSIM:0.694034080945
TV1-128 PSNR:28.223695164 SSIM:0.856850820188
TV1-256 PSNR:28.5347079368 SSIM:0.867740083569
TV1-512 PSNR:28.5940769615 SSIM:0.868931446361
step 1
FBP-64 PSNR:21.428297634 SSIM:0.53347788072
FBP-128 PSNR:26.4649257858 SSIM:0.724104185116
FBP-256 PSNR:31.471419511 SSIM:0.897979694459
FBP-512 PSNR:33.8382577325 SSIM:0.956947705496
SART-64 PSNR:25.1373299011 SSIM:0.824133472063
SART-128 PSNR:27.2464459336 SSIM:0.885656973691
SART-256 PSNR:28.8490102634 SSIM:0.928414322417
SART-512 PSNR:29.7165716666 SSIM:0.948166851401
TV1-64 PSNR:25.9999835317 SSIM:0.740202097627
TV1-1

FBP-64 PSNR:22.5324814136 SSIM:0.575353751397
FBP-128 PSNR:27.4815178211 SSIM:0.761406834024
FBP-256 PSNR:31.9659755997 SSIM:0.915720788139
FBP-512 PSNR:33.7866191391 SSIM:0.962292076045
SART-64 PSNR:25.2741620426 SSIM:0.839402977867
SART-128 PSNR:27.187198087 SSIM:0.893582275968
SART-256 PSNR:28.7102020394 SSIM:0.932390251511
SART-512 PSNR:29.6718756081 SSIM:0.951020432861
TV1-64 PSNR:26.9732621878 SSIM:0.77822919289
TV1-128 PSNR:30.2084547013 SSIM:0.905207338489
TV1-256 PSNR:30.5338706547 SSIM:0.913518185941
TV1-512 PSNR:30.5760373088 SSIM:0.914249356921
step 16
FBP-64 PSNR:22.5079754663 SSIM:0.577733429184
FBP-128 PSNR:27.4268114903 SSIM:0.764436969726
FBP-256 PSNR:31.8113622287 SSIM:0.917470232512
FBP-512 PSNR:33.5632904413 SSIM:0.963158585613
SART-64 PSNR:25.188559303 SSIM:0.840342373317
SART-128 PSNR:27.0520039105 SSIM:0.893925064029
SART-256 PSNR:28.5146745899 SSIM:0.932376618177
SART-512 PSNR:29.427586664 SSIM:0.950674418642
TV1-64 PSNR:26.9075847468 SSIM:0.78113467544
TV1-128 

FBP-64 PSNR:22.4433232996 SSIM:0.59258159986
FBP-128 PSNR:27.3753812358 SSIM:0.776838664173
FBP-256 PSNR:31.6424464367 SSIM:0.922903055164
FBP-512 PSNR:33.3349179962 SSIM:0.965994216349
SART-64 PSNR:24.9048221118 SSIM:0.846432105041
SART-128 PSNR:26.745498323 SSIM:0.89844514349
SART-256 PSNR:28.1610133736 SSIM:0.935770937478
SART-512 PSNR:29.0084873603 SSIM:0.952970209268
TV1-64 PSNR:26.6508310141 SSIM:0.790186797325
TV1-128 PSNR:30.0010577495 SSIM:0.911736104868
TV1-256 PSNR:30.3750076626 SSIM:0.922441572589
TV1-512 PSNR:30.416646655 SSIM:0.923097864369
step 31
FBP-64 PSNR:22.483600812 SSIM:0.59309380552
FBP-128 PSNR:27.436698306 SSIM:0.778342435346
FBP-256 PSNR:31.7088937592 SSIM:0.923997394702
FBP-512 PSNR:33.3873037234 SSIM:0.966495497296
SART-64 PSNR:24.9450960451 SSIM:0.847719859789
SART-128 PSNR:26.7935464928 SSIM:0.89971306965
SART-256 PSNR:28.2005541589 SSIM:0.936712509109
SART-512 PSNR:29.0348569034 SSIM:0.953696312191
TV1-64 PSNR:26.7045359236 SSIM:0.791277338882
TV1-128 PSN

FBP-64 PSNR:22.7479806946 SSIM:0.608907450972
FBP-128 PSNR:27.620327282 SSIM:0.793009706573
FBP-256 PSNR:31.7606402411 SSIM:0.93067811103
FBP-512 PSNR:33.4284309959 SSIM:0.968731326728
SART-64 PSNR:24.9734577 SSIM:0.854715854668
SART-128 PSNR:26.7948399106 SSIM:0.904228479228
SART-256 PSNR:28.2134938375 SSIM:0.939265100363
SART-512 PSNR:29.0851354932 SSIM:0.955066103824
TV1-64 PSNR:26.9208794208 SSIM:0.806420716523
TV1-128 PSNR:30.1426433012 SSIM:0.919217134994
TV1-256 PSNR:30.4779137366 SSIM:0.928068196413
TV1-512 PSNR:30.5181677218 SSIM:0.928717286406
step 46
FBP-64 PSNR:22.7875083168 SSIM:0.610589541624
FBP-128 PSNR:27.6811489575 SSIM:0.794090245841
FBP-256 PSNR:31.8407035651 SSIM:0.931075981822
FBP-512 PSNR:33.5133109683 SSIM:0.96878429023
SART-64 PSNR:25.0121172823 SSIM:0.855137997631
SART-128 PSNR:26.8442440901 SSIM:0.904837846199
SART-256 PSNR:28.2705035964 SSIM:0.939737394395
SART-512 PSNR:29.1442370033 SSIM:0.955436456835
TV1-64 PSNR:26.9536179146 SSIM:0.806924089384
TV1-128 P

FBP-64 PSNR:22.4325144422 SSIM:0.599402477877
FBP-128 PSNR:27.3589106571 SSIM:0.782387280653
FBP-256 PSNR:31.725037614 SSIM:0.925261460011
FBP-512 PSNR:33.6154810681 SSIM:0.967097307753
SART-64 PSNR:24.8716666212 SSIM:0.845189482961
SART-128 PSNR:26.7564935637 SSIM:0.897271203625
SART-256 PSNR:28.2849979019 SSIM:0.934933405188
SART-512 PSNR:29.2684175954 SSIM:0.952667631517
TV1-64 PSNR:26.5441548517 SSIM:0.792125166363
TV1-128 PSNR:29.9316434119 SSIM:0.911188482753
TV1-256 PSNR:30.2993570518 SSIM:0.921849169775
TV1-512 PSNR:30.3428005206 SSIM:0.922622623674
step 61
FBP-64 PSNR:22.4703848399 SSIM:0.601486380293
FBP-128 PSNR:27.3984069874 SSIM:0.783938557977
FBP-256 PSNR:31.7484417923 SSIM:0.925892999794
FBP-512 PSNR:33.6182713544 SSIM:0.967282184016
SART-64 PSNR:24.9087012693 SSIM:0.846128937905
SART-128 PSNR:26.7922361553 SSIM:0.898031493655
SART-256 PSNR:28.3132938861 SSIM:0.935482196547
SART-512 PSNR:29.2871899329 SSIM:0.953075341153
TV1-64 PSNR:26.5767894666 SSIM:0.793503464461
TV1-

...

FBP-64 PSNR:21.048728782 SSIM:0.539127959408
FBP-128 PSNR:26.1042017856 SSIM:0.72296524194
FBP-256 PSNR:31.3764009334 SSIM:0.885471497081
FBP-512 PSNR:34.520558766 SSIM:0.943225527617
SART-64 PSNR:24.5987736295 SSIM:0.815592875105
SART-128 PSNR:26.7808240904 SSIM:0.868726854878
SART-256 PSNR:28.8637839866 SSIM:0.909834400935
SART-512 PSNR:30.5477789707 SSIM:0.93217994533
TV1-64 PSNR:25.2916134145 SSIM:0.737920815406
TV1-128 PSNR:29.3195562558 SSIM:0.877174930392
TV1-256 PSNR:29.9304103676 SSIM:0.892627845753
TV1-512 PSNR:29.9936716424 SSIM:0.893482444678
step 414
FBP-64 PSNR:21.0484266141 SSIM:0.539182670089
FBP-128 PSNR:26.1046839477 SSIM:0.723012054437
FBP-256 PSNR:31.3769022741 SSIM:0.88551790922
FBP-512 PSNR:34.5211020734 SSIM:0.943281756422
SART-64 PSNR:24.5976467353 SSIM:0.815708882087
SART-128 PSNR:26.7782050802 SSIM:0.868824676803
SART-256 PSNR:28.8589571854 SSIM:0.909913201243
SART-512 PSNR:30.5406014994 SSIM:0.932251059724
TV1-64 PSNR:25.2895759039 SSIM:0.738030116124
TV1-128

FBP-64 PSNR:21.0639478798 SSIM:0.540157720845
FBP-128 PSNR:26.1130708342 SSIM:0.7235910553
FBP-256 PSNR:31.3632878994 SSIM:0.885827711113
FBP-512 PSNR:34.4863126413 SSIM:0.943721407689
SART-64 PSNR:24.6021495036 SSIM:0.815842442847
SART-128 PSNR:26.7655129065 SSIM:0.868759137503
SART-256 PSNR:28.8249376426 SSIM:0.909868172756
SART-512 PSNR:30.4851661121 SSIM:0.932388397888
TV1-64 PSNR:25.3057863435 SSIM:0.73881944462
TV1-128 PSNR:29.3202194785 SSIM:0.877424368196
TV1-256 PSNR:29.9282679946 SSIM:0.892768296728
TV1-512 PSNR:29.9910324829 SSIM:0.893619787647
step 429
FBP-64 PSNR:21.0784672194 SSIM:0.540356392211
FBP-128 PSNR:26.1311846782 SSIM:0.723876589879
FBP-256 PSNR:31.3887448519 SSIM:0.886026684688
FBP-512 PSNR:34.5147952823 SSIM:0.943811452232
SART-64 PSNR:24.6179532641 SSIM:0.816100237798
SART-128 PSNR:26.7858038364 SSIM:0.868997737141
SART-256 PSNR:28.8518351763 SSIM:0.910053574232
SART-512 PSNR:30.5196676929 SSIM:0.932534755333
TV1-64 PSNR:25.3256526606 SSIM:0.739094340941
TV1-1

FBP-64 PSNR:21.0684189514 SSIM:0.539780670554
FBP-128 PSNR:26.1297954533 SSIM:0.723505709279
FBP-256 PSNR:31.4150354199 SSIM:0.886453869626
FBP-512 PSNR:34.5902729947 SSIM:0.944622022615
SART-64 PSNR:24.614901193 SSIM:0.816492879112
SART-128 PSNR:26.7894385621 SSIM:0.869648993291
SART-256 PSNR:28.8667831794 SSIM:0.910929249838
SART-512 PSNR:30.5496297334 SSIM:0.933499111523
TV1-64 PSNR:25.3120371861 SSIM:0.738780982472
TV1-128 PSNR:29.3585581925 SSIM:0.878428781216
TV1-256 PSNR:29.9745824079 SSIM:0.893932627344
TV1-512 PSNR:30.0380118266 SSIM:0.894783602722
step 444
FBP-64 PSNR:21.0767202054 SSIM:0.540195276131
FBP-128 PSNR:26.1401941231 SSIM:0.723852103446
FBP-256 PSNR:31.4257203187 SSIM:0.886631864976
FBP-512 PSNR:34.5986021515 SSIM:0.944713155254
SART-64 PSNR:24.6208162798 SSIM:0.816694288572
SART-128 PSNR:26.7979264122 SSIM:0.869818234141
SART-256 PSNR:28.877757601 SSIM:0.911059971572
SART-512 PSNR:30.5621819978 SSIM:0.933605033492
TV1-64 PSNR:25.3206035466 SSIM:0.739132385684
TV1-

FBP-64 PSNR:21.0763274118 SSIM:0.539109667122
FBP-128 PSNR:26.1451713377 SSIM:0.722999085963
FBP-256 PSNR:31.4499470754 SSIM:0.886563456033
FBP-512 PSNR:34.6462950234 SSIM:0.944854619982
SART-64 PSNR:24.6469558456 SSIM:0.816760743292
SART-128 PSNR:26.8322917332 SSIM:0.870051627838
SART-256 PSNR:28.9256622484 SSIM:0.911372320685
SART-512 PSNR:30.6245620168 SSIM:0.933955219923
TV1-64 PSNR:25.3287524045 SSIM:0.738407360221
TV1-128 PSNR:29.3891678773 SSIM:0.878752089123
TV1-256 PSNR:30.0104269334 SSIM:0.894258418758
TV1-512 PSNR:30.0741564207 SSIM:0.895108668728
step 459
FBP-64 PSNR:21.069090569 SSIM:0.538863628524
FBP-128 PSNR:26.1359738147 SSIM:0.72268548818
FBP-256 PSNR:31.4388076257 SSIM:0.886291006124
FBP-512 PSNR:34.6351507251 SSIM:0.944703218818
SART-64 PSNR:24.6433876505 SSIM:0.816471935756
SART-128 PSNR:26.8272333526 SSIM:0.869771069679
SART-256 PSNR:28.9197990536 SSIM:0.911129555861
SART-512 PSNR:30.6186921328 SSIM:0.9337646408
TV1-64 PSNR:25.3221689564 SSIM:0.738058584704
TV1-12

FBP-64 PSNR:21.0491110449 SSIM:0.537924354843
FBP-128 PSNR:26.1104987675 SSIM:0.721919609088
FBP-256 PSNR:31.3997230074 SSIM:0.886031702012
FBP-512 PSNR:34.5867757442 SSIM:0.944792769757
SART-64 PSNR:24.6121572522 SSIM:0.815554428529
SART-128 PSNR:26.7820794841 SSIM:0.868977472876
SART-256 PSNR:28.8609109893 SSIM:0.910566514142
SART-512 PSNR:30.5493368966 SSIM:0.933445264446
TV1-64 PSNR:25.2978723093 SSIM:0.736972701804
TV1-128 PSNR:29.3570210891 SSIM:0.878092055862
TV1-256 PSNR:29.9865426019 SSIM:0.89368678255
TV1-512 PSNR:30.0496827051 SSIM:0.894535347826
step 474
FBP-64 PSNR:21.0483738897 SSIM:0.537833601894
FBP-128 PSNR:26.1081259531 SSIM:0.721808284868
FBP-256 PSNR:31.3945895698 SSIM:0.885924725549
FBP-512 PSNR:34.5803456264 SSIM:0.944736968478
SART-64 PSNR:24.6119255549 SSIM:0.815492618984
SART-128 PSNR:26.7803183349 SSIM:0.868884682253
SART-256 PSNR:28.8575417329 SSIM:0.910462358584
SART-512 PSNR:30.5446120242 SSIM:0.933352232581
TV1-64 PSNR:25.2989796788 SSIM:0.73696481279
TV1-